<a href="https://colab.research.google.com/github/sakda1412/Satellite_ISR/blob/master/Satellite_ISR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd drive/My Drive/Satellite_ISR

In [0]:
!python setup.py install
!pip install -U PyYAML

# **Training**




### สร้างโมเดล (Model)








In [0]:
from ISR.models import RRDN
from ISR.models import Discriminator
from ISR.models import Cut_VGG19

In [0]:
lr_train_patch_size = 40
layers_to_extract = [5, 9]
scale = 6
hr_train_patch_size = lr_train_patch_size * scale

rrdn  = RRDN(arch_params={'C':4, 'D':3, 'G':64, 'G0':64, 'T':10, 'x':scale}, patch_size=lr_train_patch_size)
f_ext = Cut_VGG19(patch_size=hr_train_patch_size, layers_to_extract=layers_to_extract)
discr = Discriminator(patch_size=hr_train_patch_size, kernel_size=3)

### สร้างตัวฝึกสอน (Trainer) โดยสามารถตั้งค่าต่างๆตามที่เราต้องการให้โมเดล (Model)

In [0]:
from ISR.train import Trainer
loss_weights = {
  'generator': 1.0,
  'feature_extractor': 0.08333,
  'discriminator': 0.01
}
losses = {
  'generator': 'mae',
  'feature_extractor': 'mse',
  'discriminator': 'binary_crossentropy'
} 

log_dirs = {'logs': './logs/x6', 'weights': './weights/x6'}

learning_rate = {'initial_value': 0.0001, 'decay_factor': 0.5, 'decay_frequency': 10000}



trainer = Trainer(
    generator=rrdn,
    discriminator=discr,
    feature_extractor=f_ext,
    lr_train_dir='sen2/setUp6/LR_sub_train',
    hr_train_dir='sen2/setUp6/HR_sub_train',
    lr_valid_dir='sen2/setUp6/LR_sub_valid',
    hr_valid_dir='sen2/setUp6/HR_sub_valid',
    loss_weights=loss_weights,
    learning_rate=learning_rate,
    dataname='sen2',
    log_dirs=log_dirs,
    weights_generator=None,
    weights_discriminator=None,
    n_validation=10,
)


### เริ่มการฝึก

In [0]:
trainer.train(
    epochs=10,
    steps_per_epoch=40,
    batch_size=8,
    monitored_metrics={'val_generator_PSNR_Y': 'max'}
)

# **Prediction**

### สร้างโมเดลและโหลดค่า weights ลงโมเดล และทำการ Prediction สร้างรูปภาพความละเอียดสูง

In [0]:
from ISR.models import RRDN
from ISR.predict import Predictor
from ISR.utils import metrics
rrdn  = RRDN(arch_params={'C':4, 'D':3, 'G':64, 'G0':64, 'T':10, 'x':6}, patch_size=80)

predictor = Predictor(input_dir='sen2/setUp6/LR_sub_test')
predictor.get_predictions(model=rrdn, weights_path='weights/GEN+VGG/rrdn-C4-D3-G64-G064-T10-x6_best-val_generator_PSNR_Y_epoch1997.hdf5')

### ทำการวัดผลด้วยค่า PSNR และ SSIM

In [0]:
import os
import numpy as np
from PIL import Image
import cv2
from skimage.metrics import structural_similarity ,peak_signal_noise_ratio

In [0]:
path1='sen2/setUp6/HR_sub_test/'
path2='data/output/LR_sub_test/rrdn-C4-D3-G64-G064-T10-x6/GEN+VGG_epoch1997/'
PSNR_all = []
SSIM_all = []
GT_list = os.listdir(path1)
gen_list = os.listdir(path2)
gen=np.sort(gen_list)
GT=np.sort(GT_list)
if np.all(gen==GT):
  for i, img_path in enumerate(GT):
    img1 = cv2.imread(path1+img_path)
    img2 = cv2.imread(path2+img_path)
    psnr = peak_signal_noise_ratio(img1, img2)
    ssim = structural_similarity(img1,img2,multichannel=True)
    print(img_path +'  PSNR: {:.6f} dB, SSIM: {:.6f}'.format(psnr,ssim))
    PSNR_all.append(psnr)
    SSIM_all.append(ssim)


print('Average: PSNR: {:.6f} dB, SSIM: {:.6f}'.format(
        sum(PSNR_all) / len(PSNR_all),
        sum(SSIM_all) / len(SSIM_all)))